## SDXL Resource Lists

**Homepage :** [SD Webui Complete setup](https://github.com/ffxvs/sd-webui-complete-setup)  
**Guide :** [Resource Lists Guide](https://github.com/ffxvs/sd-webui-complete-setup/wiki/Resource-Lists-Guide)

## 1. Requirements

### • Initialization
**REQUIRED EVERY TIME YOU START THE MACHINE**

In [ ]:
# Civitai Token - Required to download models that require login
civitai_token = ''


################################################################################################################

import os
os.chdir('/')
from internal.main import *

apply_envs1()
completed_message()

## 2. ControlNet

### • ControlNet Models

In [ ]:
# ControlNet - https://github.com/Mikubill/sd-webui-controlnet/discussions/2039
# Select models to download. Default models are : depth, ip-adapter, openpose

controlnet = {  
    'blur': boolean[0],
    'blur_anime': boolean[0],
    'canny': boolean[0],
    'canny_anime': boolean[0],
    'depth': boolean[1],
    'depth_anime': boolean[0],
    'ipadapter': boolean[1],
    'ipadapter_vit_h': boolean[0],
    'ipadapter_plus': boolean[0],
    'ipadapter_plus_face': boolean[0],
    'lineart': boolean[0],
    'openpose': boolean[1],
    'openpose_anime': boolean[0],
    'recolor': boolean[0],
    'scribble_anime': boolean[0],
    'sketch': boolean[0],
    'softedge': boolean[0],
}


################################################################################################################

print("\n⏳ Download selected controlNet models...")
download_controlnet(controlnet, sdxl_controlnet_url, sdxl)
os.chdir(webui)
completed_message()

### • Install from URLs

In [ ]:
# Install controlNet models from URL 
other_controlnet = []


########################################################################################

if other_controlnet:
    print('⏳ Downloading controlNet models...')
    other_resources(other_controlnet, f'{controlnet_models_path}/sdxl', civitai_token)

os.chdir(webui)
completed_message()

## 3. SDXL Models
**Choose models you want to download**  
Some models hosted by Civitai require you to login to download.  
Create an API key in [Civitai user settings](https://civitai.com/user/account) then paste it to `civiai_Token` inside Initialization cell.

### • Anime / Cartoon / 3D

In [ ]:
# Run this cell to display the model list

models_selection(sdxl_anime_models_url, sdxl, civitai_token)
os.chdir(webui)

### • General Purpose

In [ ]:
# Run this cell to display the model list

models_selection(sdxl_general_models_url, sdxl, civitai_token)
os.chdir(webui)

### • Realistic

In [ ]:
# Run this cell to display the model list

models_selection(sdxl_realistic_models_url, sdxl, civitai_token)
os.chdir(webui)

### • Woman

In [ ]:
# Run this cell to display the model list

models_selection(sdxl_woman_models_url, sdxl, civitai_token)
os.chdir(webui)

### • Install from URLs

In [ ]:
# Install models from URL 
other_models = []


########################################################################################

if other_models:
    print('⏳ Downloading models...')
    other_resources(other_models, f'{models_path}/sdxl', civitai_token)

os.chdir(webui)
completed_message()

## 4. LoRA, Embedding, Upscaler and VAE

### • Built-in Resources
**This is just for information. No need to run this cell.**

**Pre-installed LoRA :**
* [Detail Tweaker XL](https://civitai.com/models/122359)

**Pre-installed Embeddings :**
* [Negative Prompts Pack XL](https://huggingface.co/ffxvs/negative-prompts-pack-xl)
* [Embeddings Collection XL](https://huggingface.co/ffxvs/embeddings-collection-xl)

**Pre-installed Upscaler :**  
* [UltraSharp](https://openmodeldb.info/models/4x-UltraSharp)

**Pre-installed VAE :**
* [sdxl-vae-fp16-fix](https://huggingface.co/madebyollin/sdxl-vae-fp16-fix)

### • Resource Lists

In [ ]:
####################################### LORA ##########################################

# Blur SDXL - https://civitai.com/models/124855
blurXL = boolean[0]

# DetailedEyes XL - https://civitai.com/models/120723
detailEyes = boolean[0]

# LCM Lora SDXL - https://huggingface.co/latent-consistency/lcm-lora-sdxl
lcmXL = boolean[0]

# LCM&TurboMix - https://civitai.com/models/216190
lcmTurbo = boolean[0]

# Hands XL - https://civitai.com/models/200255
handsXL = boolean[0]

# SOAP (Shot On A Phone) - https://civitai.com/models/131818
soap = boolean[0]

# XL More Art Full - https://civitai.com/models/124347
moreArt = boolean[0]


###################################### UPSCALER ########################################

# BSRGAN - https://openmodeldb.info/models/4x-BSRGAN
bsrgan = boolean[0]

# LSDIRPlus - https://openmodeldb.info/models/4x-LSDIRplus
lsdirPlus = boolean[0]

# NMKD Superscale - https://openmodeldb.info/models/4x-NMKD-Superscale
superscale = boolean[0]

# Nomos8kSC - https://openmodeldb.info/models/4x-Nomos8kSC
nomos8ksc = boolean[0]

# Remacri - https://openmodeldb.info/models/4x-Remacri
remacri = boolean[0]

# UltraMix - https://upscale.wiki/w/index.php?title=Model_Database&oldid=1571
ultraMix = boolean[0]

# Valar - https://openmodeldb.info/models/4x-Valar
valar = boolean[0]


###################################################################################################

os.chdir(webui)
download_builtin_resources(sdxl_builtin_resources_url, sdxl)
download_lora(sdxl_lora_list_url, sdxl)
download_upscaler()
os.chdir(webui)
completed_message()

### • Install from URLs

In [ ]:
# LoRA
other_lora = []

# Embeddings
other_embeddings = []

# Upscaler
other_upscaler = []

# VAE
other_vae = []


###################################################################################################

download_other_resources(sdxl, other_lora, other_embeddings, other_upscaler, other_vae, civitai_token)
os.chdir(webui)
completed_message()